In [4]:
from google.colab import drive
import os

# Perintah ini akan memunculkan jendela pop-up untuk meminta izin
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/Dataset-Skripsi'
dataset_dir = os.path.join(base_dir, 'Dataset')
output_dir = os.path.join(base_dir, 'Output')

# Membuat folder jika belum ada
os.makedirs(dataset_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

print("Google Drive terhubung dan folder siap.")
print(f"Folder Dataset: {dataset_dir}")
print(f"Folder Output: {output_dir}")

Mounted at /content/drive
Google Drive terhubung dan folder siap.
Folder Dataset: /content/drive/MyDrive/Dataset-Skripsi/Dataset
Folder Output: /content/drive/MyDrive/Dataset-Skripsi/Output


In [2]:
# Install MTCNN untuk deteksi wajah
!pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.1 MB/s eta 0:00:00


In [3]:
# Untuk Deep Learning (TensorFlow & Keras)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Untuk Pengolahan Angka dan Gambar
import numpy as np
import cv2  # Ini adalah OpenCV
import matplotlib.pyplot as plt

# Untuk Utilitas Sistem dan File
import os
import glob
import random

# Untuk Deteksi Wajah (yang baru kita install)
from mtcnn.mtcnn import MTCNN

# Untuk Evaluasi Model
from sklearn.metrics import classification_report, confusion_matrix

print("Semua library dasar berhasil di-impor.")

Semua library dasar berhasil di-impor.
